In [72]:
import os
import sys
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import AzureChatOpenAI
import re 
from langchain_community.graphs import Neo4jGraph
import dotenv
from langchain_community.graphs import Neo4jGraph
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
import numpy as np 
import pandas as pd
# from project import web_crawl
import subprocess
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# connection to API's 

In [73]:

#LLM API's 
dotenv.load_dotenv()
llm=AzureChatOpenAI(
    azure_deployment=os.getenv('AZURE_OPENAI_DEPLOYMENT_MODEL'),
    api_version=os.getenv('AZURE_OpenAI_API_VERSION'),
    temperature=0,
    
)

graph=Neo4jGraph(url=os.getenv('NEO4J_URI_ONLINE'),
                 username=os.getenv('NEO4J_USERNAME_ONLINE'),
                 password=os.getenv('NEO4J_PASSWORD_ONLINE')
                #  database=os.getenv('NEO4J_DATABASE')
)


In [74]:
# empty graph database
# DElete everything from the database 
# delete_cypher=""" 
# Match(n)
# DETACH DELETE n
# """
# graph.query(delete_cypher)

# extract information from weburl


In [75]:
from langchain_core.documents import Document
from helpers.clean_data import clean_text
# url_input=("https://en.wikipedia.org/wiki/Elon_Musk")
# url_input=('https://en.wikipedia.org/wiki/Mark_Zuckerberg')
# url_input=('https://en.wikipedia.org/wiki/Bill_Gates')
# url_input=('https://en.wikipedia.org/wiki/Jeff_Bezos')
# url_input=('https://en.wikipedia.org/wiki/Steve_Jobs')
# url_input=('https://en.wikipedia.org/wiki/Sam_Altman')
url_input=('https://en.wikipedia.org/wiki/Mukesh_Ambani')
loader=WebBaseLoader([url_input])
data=loader.load().pop().page_content
data
    

'\n\n\n\nMukesh Ambani - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload file\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nEarly life\n\n\n\n\n\n\n\n\n2\nEducation\n\n\n\n\n\n\n\n\n3\nCareer\n\n\n\n\n\n\n\n\n4\nBoard memberships\n\n\n\n\n\n\n\n\n5\nAwards and honors\n

In [76]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

data=clean_text(data)
documents=[Document(page_content=data)]
# print(documents)
splitter=RecursiveCharacterTextSplitter(chunk_size=3000,chunk_overlap=100)
smaller_doc=splitter.split_documents(documents)
len(smaller_doc)

10

# setting up prompt template example

In [77]:
template="""
you are an intelliegnt bot who can analyze any text with person information {doc} , 
your job is to read and analyse the information and create a json dictionary.
dictionary has the followinmg key:
Name:
Company:
Skills:
output must be json nothing else
"""

In [78]:
prompt=PromptTemplate(template=template,input_variables=['doc'])
llmchain=LLMChain(llm=llm,prompt=prompt)

In [79]:
for doc in smaller_doc[:2]:
    response=llmchain.invoke({'doc':doc.page_content})
    print(response['text'])
    

{
  "Name": "Mukesh Dhirubhai Ambani",
  "Company": "Reliance Industries",
  "Skills": ["Business Management", "Leadership", "Market Manipulation", "Political Corruption", "Cronyism", "Exploitation"]
}
{
  "Name": "Mukesh Ambani",
  "Company": "Reliance Industries",
  "Skills": [
    "Business Leadership",
    "Oil Refineries",
    "Petrochemical Products",
    "Information and Communications Technology",
    "Grassroots Petroleum Refinery",
    "Digital Infrastructure"
  ]
}


# from the predefined prompts

In [80]:
from helpers.prompts import graphPrompt
results=[]
for doc in smaller_doc[:30]:
    results.append(graphPrompt(doc.page_content))
results

{
    "entities": [
        {
            "label": "Mukesh Ambani",
            "id": "Mukesh Ambani",
            "role": "Businessman",
            "description": "Chairman and MD of Reliance Industries"
        },
        {
            "label": "Reliance Industries",
            "id": "Reliance Industries",
            "role": "Organization",
            "description": "Indian conglomerate company"
        },
        {
            "label": "Forbes",
            "id": "Forbes",
            "role": "Organization",
            "description": "American business magazine"
        },
        {
            "label": "India",
            "id": "India",
            "role": "Country",
            "description": "South Asian country"
        },
        {
            "label": "Dhirubhai Ambani",
            "id": "Dhirubhai Ambani",
            "role": "Person",
            "description": "Mukesh Ambani's father and founder of Reliance Industries"
        },
        {
            "label": "Kokil

[{'entities': [{'label': 'Mukesh Ambani',
    'id': 'Mukesh Ambani',
    'role': 'Businessman',
    'description': 'Chairman and MD of Reliance Industries'},
   {'label': 'Reliance Industries',
    'id': 'Reliance Industries',
    'role': 'Organization',
    'description': 'Indian conglomerate company'},
   {'label': 'Forbes',
    'id': 'Forbes',
    'role': 'Organization',
    'description': 'American business magazine'},
   {'label': 'India',
    'id': 'India',
    'role': 'Country',
    'description': 'South Asian country'},
   {'label': 'Dhirubhai Ambani',
    'id': 'Dhirubhai Ambani',
    'role': 'Person',
    'description': "Mukesh Ambani's father and founder of Reliance Industries"},
   {'label': 'Kokilaben Ambani',
    'id': 'Kokilaben Ambani',
    'role': 'Person',
    'description': "Mukesh Ambani's mother"},
   {'label': 'Anil Ambani',
    'id': 'Anil Ambani',
    'role': 'Person',
    'description': "Mukesh Ambani's younger brother"}],
  'relationships': [{'label': 'Founded

In [81]:
for res in results:
    print(res)


{'entities': [{'label': 'Mukesh Ambani', 'id': 'Mukesh Ambani', 'role': 'Businessman', 'description': 'Chairman and MD of Reliance Industries'}, {'label': 'Reliance Industries', 'id': 'Reliance Industries', 'role': 'Organization', 'description': 'Indian conglomerate company'}, {'label': 'Forbes', 'id': 'Forbes', 'role': 'Organization', 'description': 'American business magazine'}, {'label': 'India', 'id': 'India', 'role': 'Country', 'description': 'South Asian country'}, {'label': 'Dhirubhai Ambani', 'id': 'Dhirubhai Ambani', 'role': 'Person', 'description': "Mukesh Ambani's father and founder of Reliance Industries"}, {'label': 'Kokilaben Ambani', 'id': 'Kokilaben Ambani', 'role': 'Person', 'description': "Mukesh Ambani's mother"}, {'label': 'Anil Ambani', 'id': 'Anil Ambani', 'role': 'Person', 'description': "Mukesh Ambani's younger brother"}], 'relationships': [{'label': 'Founded', 'id': 'relationship1', 'type': 'Founded', 'source': 'Dhirubhai Ambani', 'target': 'Reliance Industries

In [82]:
# empty neo
#graph.query(delete_cypher)

# inject data into neo4j

In [83]:
from neo4j import GraphDatabase


In [84]:
driver=GraphDatabase.driver(uri=os.getenv('NEO4J_URI_ONLINE'),auth=(os.getenv('NEO4J_USERNAME_ONLINE'),os.getenv('NEO4J_PASSWORD_ONLINE')))
existing_set=[]
def sanitize_label(label):
    sanitized=re.sub(r"[^\w]", "_", label)
    return re.sub(r"_+", "_", sanitized).strip("_")

def insert_entities(session,entities):
    for entity in entities:
        
        sanitize_labels=sanitize_label(entity['id'])
        query=f"""
            MERGE (n:{sanitize_labels} {{id:$id}})
            ON CREATE SET
                n.role=$role,
                n.description=$description
            ON MATCH SET
                n.role=$role,
                n.description=$description

            """
        session.run(query,label=entity['label'], id=entity['id'],role=entity['role'],description=entity['label'])
        print(f'Inserted entity: {entity["id"]}')
        existing_set.append(entity['label'])
    
def insert_relationship(session,relationships):
    for relation in relationships:
        query = f"""
        MATCH (a {{id: $source}}), (b {{id: $target}})
        MERGE (a)-[:{sanitize_label(relation['type'])} {{
            id: $id,
            description: $description
        }}]->(b)
        """
        session.run(query,source=relation['source'],target=relation['target'],id=relation['id'],description=relation['description'])
        print(f'Inserted relationship: {relation["id"]}')


In [85]:
with driver.session() as session:
    for result in results:
        try:
            session.execute_write(insert_entities,result['entities'])
            session.execute_write(insert_relationship,result['relationships'])
        except Exception as e:
            print(f'Error {e}')
driver.close()

Inserted entity: Mukesh Ambani
Inserted entity: Reliance Industries
Inserted entity: Forbes
Inserted entity: India
Inserted entity: Dhirubhai Ambani
Inserted entity: Kokilaben Ambani
Inserted entity: Anil Ambani
Inserted relationship: relationship1
Inserted relationship: relationship2
Inserted entity: Mukesh Ambani
Inserted entity: Reliance Industries
Inserted entity: Sea Wind
Inserted entity: Hill Grange High School
Inserted entity: St. Xavier's College
Inserted entity: Institute of Chemical Technology
Inserted entity: Patalganga Petrochemical Plant
Inserted entity: Jamnagar Refinery
Inserted entity: Bharti Airtel
Inserted entity: Bank of America
Inserted relationship: relationship1
Inserted relationship: relationship2
Inserted relationship: relationship3
Inserted relationship: relationship4
Inserted entity: Mukesh Ambani
Inserted entity: Bank of America
Inserted entity: Forbes
Inserted entity: Alibaba Group
Inserted entity: Indian Premier League
Inserted entity: Indian Super League
I